In [1]:
import os
import numpy
import numpy as np
import sklearn
import pandas as pd
import kagglehub


C:\Users\manas\Projects\ML-Network-Detection\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="sklearn")

In [3]:
data_path = kagglehub.dataset_download("hassan06/nslkdd")

train_data_path = os.path.join(data_path, "KDDTrain+.txt")
test_data_path = os.path.join(data_path, "KDDTest+.txt")

In [4]:
from data_loader import Dataset

dataset = Dataset(train_data_path, test_data_path)

columns = [
    'duration', 'protocol_type', 'service', 'flag', 'src_bytes',
    'dst_bytes', 'land', 'wrong_fragment', 'urgent', 'hot',
    'num_failed_logins', 'logged_in', 'num_compromised', 'root_shell',
    'su_attempted', 'num_root', 'num_file_creations', 'num_shells',
    'num_access_files', 'num_outbound_cmds', 'is_host_login',
    'is_guest_login', 'count', 'srv_count', 'serror_rate',
    'srv_serror_rate', 'rerror_rate', 'srv_rerror_rate', 'same_srv_rate',
    'diff_srv_rate', 'srv_diff_host_rate', 'dst_host_count',
    'dst_host_srv_count', 'dst_host_same_srv_rate', 'dst_host_diff_srv_rate',
    'dst_host_same_src_port_rate', 'dst_host_srv_diff_host_rate',
    'dst_host_serror_rate', 'dst_host_srv_serror_rate', 'dst_host_rerror_rate',
    'dst_host_srv_rerror_rate', 'label', 'difficulty'
]

dataset.set_columns(columns)
dataset.load_data()

df = dataset.train_df
test_df = dataset.test_df

Loaded training data: (125973, 43)
Loaded test data: (22544, 43)


In [5]:
df.head(20)

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,label,difficulty
0,0,tcp,ftp_data,SF,491,0,0,0,0,0,...,0.17,0.03,0.17,0.00,0.00,0.00,0.05,0.00,normal,20
1,0,udp,other,SF,146,0,0,0,0,0,...,0.00,0.60,0.88,0.00,0.00,0.00,0.00,0.00,normal,15
2,0,tcp,private,S0,0,0,0,0,0,0,...,0.10,0.05,0.00,0.00,1.00,1.00,0.00,0.00,neptune,19
3,0,tcp,http,SF,232,8153,0,0,0,0,...,1.00,0.00,0.03,0.04,0.03,0.01,0.00,0.01,normal,21
4,0,tcp,http,SF,199,420,0,0,0,0,...,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,normal,21
5,0,tcp,private,REJ,0,0,0,0,0,0,...,0.07,0.07,0.00,0.00,0.00,0.00,1.00,1.00,neptune,21
6,0,tcp,private,S0,0,0,0,0,0,0,...,0.04,0.05,0.00,0.00,1.00,1.00,0.00,0.00,neptune,21
7,0,tcp,private,S0,0,0,0,0,0,0,...,0.06,0.07,0.00,0.00,1.00,1.00,0.00,0.00,neptune,21
8,0,tcp,remote_job,S0,0,0,0,0,0,0,...,0.09,0.05,0.00,0.00,1.00,1.00,0.00,0.00,neptune,21
9,0,tcp,private,S0,0,0,0,0,0,0,...,0.05,0.06,0.00,0.00,1.00,1.00,0.00,0.00,neptune,21


In [6]:
X_df, y_true = dataset.get_X_y(df, label_column="label", drop_cols=["difficulty"])

In [7]:
print("X shape:", X_df.shape)
print("y shape:", y_true.shape)

X shape: (125973, 41)
y shape: (125973,)


In [8]:
from encoder import OneHotEncoderWrapper

one_hot_encoder = OneHotEncoderWrapper(categories="auto", drop=None, sparse_output=False, handle_unknown="ignore")

categorical_columns_names = ["protocol_type", "service", "flag"]

one_hot_encoder.fit(X_df, categorical_columns_names)
X_train_df = one_hot_encoder.transform(X_df)

print("Transformed X shape:", X_train_df.shape)

Fitted encoder on 3 categorical columns
Created 84 one-hot encoded features
Transformed X shape: (125973, 122)


In [9]:
from encoder import LabelEncoderWrapper

label_encoder = LabelEncoderWrapper()

label_encoder.fit(y_true)

Fitted LabelEncoder on 23 classes
Classes: ['back', 'buffer_overflow', 'ftp_write', 'guess_passwd', 'imap', 'ipsweep', 'land', 'loadmodule', 'multihop', 'neptune', 'nmap', 'normal', 'perl', 'phf', 'pod', 'portsweep', 'rootkit', 'satan', 'smurf', 'spy', 'teardrop', 'warezclient', 'warezmaster']


LabelEncoderWrapper(fitted=True, num_classes=23)

In [10]:
y_true.head(10)

0     normal
1     normal
2    neptune
3     normal
4     normal
5    neptune
6    neptune
7    neptune
8    neptune
9    neptune
Name: label, dtype: str

In [11]:
y_true_encoded = label_encoder.transform(y_true)
y_true_encoded.shape


(125973,)

In [12]:
X_train_final_df, X_val_final_df, y_train_encoded, y_val_encoded = sklearn.model_selection.train_test_split(
    X_train_df,
    y_true_encoded,
    test_size=0.25,
    random_state=35673,
    stratify=y_true_encoded
)

print("X_train_final_df shape:", X_train_final_df.shape)
print("X_val_final_df shape:", X_val_final_df.shape)
print("y_train_encoded shape:", y_train_encoded.shape)
print("y_val_encoded shape:", y_val_encoded.shape)

X_train_final_df shape: (94479, 122)
X_val_final_df shape: (31494, 122)
y_train_encoded shape: (94479,)
y_val_encoded shape: (31494,)


In [13]:
from model import RandomForestModel

random_forest_classifier = RandomForestModel(class_weight="balanced", random_state=875636, n_jobs=4, verbose=0)

In [14]:
# random_forest_classifier.set_params(n_jobs=4, verbose=0)

In [15]:
random_forest_classifier.get_params()

{'n_estimators': 100,
 'criterion': 'gini',
 'max_depth': None,
 'min_samples_split': 2,
 'min_samples_leaf': 1,
 'max_features': 'sqrt',
 'bootstrap': True,
 'class_weight': 'balanced',
 'random_state': 875636,
 'n_jobs': 4,
 'verbose': 0}

In [19]:
hyperparameter_choices = {
    'n_estimators': (50, 500),
    'criterion': ['gini', 'entropy'],
    'max_depth': (10, 50),
    'min_samples_split': (2, 20),
    'min_samples_leaf': (1, 10),
    'max_features': ['sqrt', 'log2'],
    'bootstrap': True,
    'class_weight': 'balanced',
    'random_state': 355677,
    'n_jobs': 4,
    'verbose': 0
}


In [20]:
hyperparameter_results = random_forest_classifier.tune_hyperparameters(X_train=X_train_final_df, y_train=y_train_encoded, X_val=X_val_final_df,
                                              y_val=y_val_encoded, param_distributions=hyperparameter_choices,
                                              n_trials=50, metric="macro_f1")

[I 2026-02-02 03:04:00,804] A new study created in memory with name: no-name-75bb1b7f-18eb-4ce4-b751-8b9a8f73b4a0


Starting hyperparameter tuning with 50 trials...
Optimizing for: macro_f1


Best trial: 0. Best value: 0.7319:   2%|▏         | 1/50 [00:08<07:09,  8.77s/it]

[I 2026-02-02 03:04:09,560] Trial 0 finished with value: 0.7318996858264397 and parameters: {'n_estimators': 352, 'criterion': 'entropy', 'max_depth': 26, 'min_samples_split': 4, 'min_samples_leaf': 7, 'max_features': 'log2'}. Best is trial 0 with value: 0.7318996858264397.


Best trial: 1. Best value: 0.791255:   4%|▍         | 2/50 [00:13<05:19,  6.66s/it]

...

[I 2026-02-02 03:14:54,769] Trial 47 finished with value: 0.7822277624398919 and parameters: {'n_estimators': 370, 'criterion': 'gini', 'max_depth': 36, 'min_samples_split': 18, 'min_samples_leaf': 8, 'max_features': 'sqrt'}. Best is trial 9 with value: 0.866919206064031.


Best trial: 9. Best value: 0.866919:  98%|█████████▊| 49/50 [11:02<00:13, 13.84s/it]

[I 2026-02-02 03:15:03,360] Trial 48 finished with value: 0.7684360015342455 and parameters: {'n_estimators': 202, 'criterion': 'entropy', 'max_depth': 14, 'min_samples_split': 20, 'min_samples_leaf': 7, 'max_features': 'sqrt'}. Best is trial 9 with value: 0.866919206064031.


Best trial: 9. Best value: 0.866919: 100%|██████████| 50/50 [11:16<00:00, 13.54s/it]

[I 2026-02-02 03:15:17,788] Trial 49 finished with value: 0.7947448171511151 and parameters: {'n_estimators': 446, 'criterion': 'gini', 'max_depth': 28, 'min_samples_split': 5, 'min_samples_leaf': 5, 'max_features': 'log2'}. Best is trial 9 with value: 0.866919206064031.

Best macro_f1: 0.8669
Best hyperparameters:
  n_estimators: 213
  criterion: gini
  max_depth: 27
  min_samples_split: 15
  min_samples_leaf: 6
  max_features: sqrt


In [21]:
hyperparameter_results

{'best_params': {'n_estimators': 213,
  'criterion': 'gini',
  'max_depth': 27,
  'min_samples_split': 15,
  'min_samples_leaf': 6,
  'max_features': 'sqrt'},
 'best_score': 0.866919206064031,
 'study': <optuna.study.study.Study at 0x202e46945c0>}

In [27]:
from optuna.visualization import plot_optimization_history,plot_param_importances,plot_contour,plot_slice,plot_parallel_coordinate
import plotly

study = hyperparameter_results['study']

In [33]:
print(plotly.__version__)

6.5.2


In [ ]:
fig = plot_param_importances(study)
fig.show()